In [1]:
!pip install transformers
!pip install tensorflow-gpu
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [44 lines of output]
  Traceback (most recent call last):
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_v

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'



In [2]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

eos_token = tokenizer.sep_token_id
pad_token = tokenizer.pad_token_id

In [3]:
input_sequences = []
target_tokens = []
max_length = 64

with open("dataset.txt", "r", encoding="utf-8") as file:
    for line in file:
        question, answer = line.strip().split('\t')

        # Encode the texts. Adding the EOS token is important for GPT to know when a sentence ends
        question_tokens = tokenizer.encode(question, add_special_tokens=False)
        answer_tokens = tokenizer.encode(answer, add_special_tokens=False) + [eos_token] # Ensure EOS token is added to the answer

        # Construct sequences of max_length or shorter
        for i in range(0, len(answer_tokens)):
            # Input sequence: question + end token + part of the answer
            seq = question_tokens + [eos_token] + answer_tokens[:i]
            if len(seq) > max_length:
                break

            # Ensure sequences are padded to have uniform length
            input_sequences.append(seq + [pad_token] * (max_length - len(seq)))  # Padding

            # Target token is the next token in the answer
            target_tokens.append(answer_tokens[i])

In [4]:
for seq in input_sequences[:10]:
    #print(seq)
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))
    print('\n')
          
for seq in target_tokens[:10]:
    print(tokenizer.decode(np.array(seq), clean_up_tokenization_spaces=True))

hi, how are you doing? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


hi, how are you doing? [SEP] i'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [5]:
# convert to TensorFlow tensors
input_sequences = tf.constant(input_sequences, dtype=tf.int16)
target_tokens = tf.constant(target_tokens, dtype=tf.int16)
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_tokens))
dataset = dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
# Create the model

In [7]:
vocab_size = tokenizer.vocab_size
embedding_dim = 256  # Size of the embedding vectors
rnn_units = 512  # Number of units in the LSTM layer

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True),
    LSTM(rnn_units),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [8]:
if tf.test.gpu_device_name():
    print(f"Default GPU Device: {tf.test.gpu_device_name()}")
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [9]:
epochs = 3

# Train the model
history = model.fit(dataset, epochs=epochs)

Epoch 1/3

1139/1139 [==============================] - 357s 311ms/step - loss: 5.1314
Epoch 2/3
1139/1139 [==============================] - 364s 320ms/step - loss: 3.9343
Epoch 3/3
1139/1139 [==============================] - 366s 321ms/step - loss: 3.4799


In [10]:
model_save_path = "my_rnn_model"
model.save(model_save_path)

INFO:tensorflow:Assets written to: my_rnn_model\assets


INFO:tensorflow:Assets written to: my_rnn_model\assets


In [11]:
from tensorflow.keras.models import load_model

model_save_path = "my_rnn_model"
loaded_model = load_model(model_save_path)

In [12]:
def generate_until_sep_token(starting_text, max_tokens=50):
    text = starting_text
    for _ in range(max_tokens):
        # Tokenize the current text
        token_ids = tokenizer.encode(text, add_special_tokens=False) + [tokenizer.sep_token_id]
        
        # Ensure the sequence is within the model's acceptable input range
        token_ids = token_ids[-(max_length-1):]
        
        # Predict the next token ID
        input_sequence = tf.constant([token_ids], dtype=tf.int16)
        predictions = loaded_model.predict(input_sequence)
        predicted_token_id = tf.argmax(predictions, axis=-1).numpy()[0]
        
        # Break if the [SEP] token is predicted
        if predicted_token_id == tokenizer.sep_token_id:
            break
        
        # Append the predicted token to the text and continue
        predicted_token = tokenizer.decode([predicted_token_id])
        text += predicted_token
        
    return text

In [13]:
starting_text = "hello, how are you?"
generated_text = generate_until_sep_token(starting_text, max_tokens=50)
print("Generated text:", generated_text)

1/1 [==============================] - 0s 22ms/step
Generated text: hello, how are you?iitiitbutbutbutandandandandandandandandandandandandandandandandandandandandandandandandandandandandandiiiiiiiiiiiiii
